# Data Preprocessing

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

from scipy.stats import norm
from scipy import stats


In [17]:
# read csv file
df=pd.read_csv("CarsRatings.csv" , index_col="Unnamed: 0")
df.head() 


,Rate,Adult Occupant,Child Occupant,Vulnerable Road Users,Safety Assist,Tested Model,Body Type,Year Of Publication,Kerb Weight,VIN From Which Rating Applies,...,Isofix/i-Size-Rear,Integrated Child Seat-Rear,Airbag cut-off switch-Rear,Seatbelt Reminder-Rear,Active Bonnet,AEB Vulnerable Road Users,AEB Pedestrian - Reverse,AEB Car-to-Car,Speed Assistance,Lane Assist System
1,5,84%,89%,81%,81%,"Jeep Grand Cherokee 2.0 PHEV 'Overland', LHD",- 5 door SUV,2022,2532kg,- all Grand Cherokees,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard
2,5,87%,87%,68%,88%,"CHERY OMODA5 1.6T, LHD",- SUV,2022,1423kg,- all OMODA5,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard
3,4,75%,84%,70%,69%,"Ford Puma 1.0 MHEV, 'Titanium', LHD",- 5 door SUV,2022,1334kg,- all Ford Pumas,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard
4,5,84%,90%,74%,84%,VW Amarok 2.0/Ford Ranger 3.0,- Pick-Up,2022,2434kg,- all double cabin Rangers,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard,Fitted_Standard
5,5,84%,89%,71%,85%,Discovery Sport,- 5 door SUV,2022,2123kg,- SALCP2FX2PH325732,...,Fitted_Standard,Not_Available,Not_Applicable,Fitted_Standard,Fitted_Standard,Fitted_Standard,Not_Available,Fitted_Standard,Fitted_Standard,Fitted_Standard


In [32]:
# TO-DO: remove  '%' in [Adult Occupant	Child Occupant,	Vulnerable Road Users,	Safety Assist] and change dtype to int
cols=['Adult Occupant', 'Child Occupant', 'Vulnerable Road Users', 'Safety Assist']

df['Adult Occupant']=df['Adult Occupant'].replace('%','')
df['Child Occupant']=df['Child Occupant'].replace('%','')
df['Safety Assist']=df['Safety Assist'].str.replace('%','')
df['Vulnerable Road Users']=df['Vulnerable Road Users'].str.replace('%','')
df['Kerb Weight']=df['Kerb Weight'].str.replace('kg','')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 1 to 311
Data columns (total 53 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Rate                             311 non-null    int64 
 1   Adult Occupant                   311 non-null    int32 
 2   Child Occupant                   311 non-null    int32 
 3   Vulnerable Road Users            218 non-null    object
 4   Safety Assist                    311 non-null    object
 5   Tested Model                     311 non-null    object
 6   Body Type                        311 non-null    object
 7   Year Of Publication              311 non-null    int64 
 8   Kerb Weight                      311 non-null    object
 9   VIN From Which Rating Applies    311 non-null    object
 10  Class                            311 non-null    object
 11  Front Airbag-Driver              311 non-null    object
 12  Belt Pretensioner-Driver         311

In [38]:
#convert the columns to int
df['Adult Occupant'] = df['Adult Occupant'].astype(str).astype(int)
df['Child Occupant'] = df['Child Occupant'].astype(str).astype(int)
df['Vulnerable Road Users'] = df['Vulnerable Road Users'].astype(str).astype(int)
df['Safety Assist'] = df['Safety Assist'].astype(str).astype(int)
df['Kerb Weight'] = df['Kerb Weight'].astype(str).astype(int)
df.info()

ValueError: invalid literal for int() with base 10: 'nan'

### dealing with Missing values

In [34]:
# replace every nan as string to np.nan
df.replace("nan",np.nan,inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 1 to 311
Data columns (total 53 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Rate                             311 non-null    int64 
 1   Adult Occupant                   311 non-null    int32 
 2   Child Occupant                   311 non-null    int32 
 3   Vulnerable Road Users            218 non-null    object
 4   Safety Assist                    311 non-null    object
 5   Tested Model                     311 non-null    object
 6   Body Type                        311 non-null    object
 7   Year Of Publication              311 non-null    int64 
 8   Kerb Weight                      311 non-null    object
 9   VIN From Which Rating Applies    311 non-null    object
 10  Class                            311 non-null    object
 11  Front Airbag-Driver              311 non-null    object
 12  Belt Pretensioner-Driver         311

In [35]:
# display all cols has a nan values
missing=df.columns[df.isnull().any()]
print(missing)

Index(['Vulnerable Road Users', 'Centre Airbag-Driver',
       'Centre Airbag-Passenger', 'Centre Airbag-Rear',
       'AEB Vulnerable Road Users', 'AEB Pedestrian - Reverse',
       'AEB Car-to-Car'],
      dtype='object')


In [36]:
# display how many missing values in missing cols
mis=df[missing].isnull().sum().sort_values(ascending=False)
print(mis)


Centre Airbag-Driver         185
Centre Airbag-Passenger      185
Centre Airbag-Rear           185
AEB Vulnerable Road Users    184
AEB Pedestrian - Reverse     184
AEB Car-to-Car               184
Vulnerable Road Users         93
dtype: int64


In [37]:
# display the percentage of missing values in missing cols
df[missing].isnull().sum()/df.shape[0]

Vulnerable Road Users        0.299035
Centre Airbag-Driver         0.594855
Centre Airbag-Passenger      0.594855
Centre Airbag-Rear           0.594855
AEB Vulnerable Road Users    0.591640
AEB Pedestrian - Reverse     0.591640
AEB Car-to-Car               0.591640
dtype: float64

since in previous step all percentages less than  > .9 none of the cols will be removed

### Dealing with Categorical Columns

In [38]:

# transform object categories to numeric categories

dfCopy=df.copy()
cat_columns = df.select_dtypes(['object']).columns # all cols has dtypy == object
#make an instance of Label Encoder 
ordinal_encoder = OrdinalEncoder()
dfCopy[cat_columns] = ordinal_encoder.fit_transform(df[cat_columns]) 
dfCopy[cat_columns]


,Adult Occupant,Child Occupant,Vulnerable Road Users,Safety Assist,Tested Model,Body Type,Kerb Weight,VIN From Which Rating Applies,Class,Front Airbag-Driver,...,Isofix/i-Size-Rear,Integrated Child Seat-Rear,Airbag cut-off switch-Rear,Seatbelt Reminder-Rear,Active Bonnet,AEB Vulnerable Road Users,AEB Pedestrian - Reverse,AEB Car-to-Car,Speed Assistance,Lane Assist System
1,22.0,34.0,34.0,44.0,97.0,11.0,210.0,139.0,4.0,0.0,...,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,25.0,32.0,21.0,50.0,29.0,25.0,65.0,199.0,10.0,0.0,...,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,13.0,29.0,23.0,31.0,66.0,11.0,48.0,125.0,12.0,0.0,...,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,22.0,35.0,27.0,47.0,221.0,22.0,202.0,280.0,6.0,0.0,...,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
5,22.0,34.0,24.0,48.0,47.0,11.0,181.0,14.0,4.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,31.0,29.0,NaN,22.0,190.0,11.0,39.0,60.0,11.0,0.0,...,0.0,3.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0,3.0
308,34.0,29.0,NaN,30.0,232.0,28.0,92.0,247.0,10.0,0.0,...,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,1.0
309,18.0,19.0,NaN,3.0,203.0,29.0,217.0,64.0,12.0,0.0,...,0.0,3.0,0.0,0.0,1.0,NaN,NaN,NaN,3.0,2.0
310,23.0,19.0,NaN,10.0,203.0,14.0,217.0,64.0,12.0,0.0,...,0.0,3.0,0.0,0.0,1.0,NaN,NaN,NaN,3.0,2.0


In [41]:
dfCopy["Integrated Child Seat-Rear"].unique()

array([3., 0., 1., 4., 2.])